In [27]:
from dotenv import load_dotenv
from openai import OpenAI

In [28]:
load_dotenv()
client = OpenAI()

In [29]:
question = input('사용자 요청 메시지')
response = client.chat.completions.create(
	model='gpt-4.1-mini',
	messages=[
		# AI 가 할 일
		{"role":"system","content":"You are a helpful assistant.using locale language."},
		# 사용자 요청
		{"role":"user","content":question}
	],
)
print(f'💬AI: {response.choices[0].message.content}')

💬AI: 지금 제가 시간을 직접 확인할 수는 없지만, 사용 중인 기기나 컴퓨터에서 시계를 확인해 보세요. 도움이 필요하시면 말씀해 주세요!


In [31]:
# 실시간 데이터는 LLM 의 학습 내용이 아닙니다.
# 	ㄴ 함수 또는 tools 를 사용해서 답변을 할 수 있는 기능이 있습니다.

# 시간 구하기 함수
from datetime import datetime

def get_currnt_time():
	now = datetime.now().strftime('%H:%M:%S')
	print(f'log : {now}')
	return now
get_currnt_time()
# 날짜 구하기 함수 : 예정		'%y-%m-%d'
def get_currnt_date():
	now = datetime.now().strftime('%y-%m-%d')
	print(f'🔄log get_current_date: {now}')
	return now

# gpt 가 사용할 함수 목록을 정의
myfunction = [
	{
		"name":"get_currnt_time",
		"description":"현재 시간 출력합니다. 포맷 HH:MM:SS 입니다."
	},
	{
		"name":"get_currnt_date",
		"description":"현재 날짜 출력합니다. 포맷 YYYY-MM-DD 입니다."
	}
]

log : 13:43:46


In [32]:
question = '현재 날짜를 알려줘'		# '현재 시간을 알려줘'
response = client.chat.completions.create(
	model='gpt-4.1-mini',
	messages=[
		{"role":"system","content":"You are a helpful assistant.using locale language."},
		{"role":"user","content":question}
	],
	functions=myfunction
)
print(f'🔄log response : {response.model_dump_json(indent=2)}')

🔄log response : {
  "id": "chatcmpl-CYm8dQJe1YB6eoqGcp0rAAr8JKaEk",
  "choices": [
    {
      "finish_reason": "function_call",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "refusal": null,
        "role": "assistant",
        "annotations": [],
        "audio": null,
        "function_call": {
          "arguments": "{}",
          "name": "get_currnt_date"
        },
        "tool_calls": null
      }
    }
  ],
  "created": 1762404231,
  "model": "gpt-4.1-mini-2025-04-14",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_4c2851f862",
  "usage": {
    "completion_tokens": 12,
    "prompt_tokens": 86,
    "total_tokens": 98,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
      "cached_tokens": 0
    }
  }
}


In [34]:

# 함수 호출을 하고 답변을 만드는 추가적인 요청이 필요합니다.
import json
# fn_name = response.choices[0].message.function_call
# ↪ FunctionCall(arguments='{}', name='get_current_time')
fn_name = getattr(response.choices[0].message.function_call,'name',None)
print(f'🔄log fn_name: {fn_name}')
if fn_name:
    func_response = globals()[fn_name]() # 문자열로 된 함수를 호출하는 방법
    followup_response = client.chat.completions.create(
        model='gpt-4.1-mini',
        messages=[
        {"role":"system","content":"You are a helpful assistant.using locale language."},
        {"role":"user",
         "content":f'{fn_name} 함수를 실행한 결과 {func_response} 이용하여 최종 응답 만들어줘.'}
        ],
        functions=myfunction
    )
    print(followup_response.model_dump_json(indent=2))
    # 함수 실행한 두번째 요청 응답
    result = followup_response.choices[0].message.content
else:
    # 첫번째 요청 응답
    result = response.choices[0].message.content    

print(f'💬AI : {result}')

🔄log fn_name: get_currnt_date
🔄log get_current_date: 25-11-06
{
  "id": "chatcmpl-CYm8viQ6SaGQTrK7UFr76GqxoTiLn",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "오늘 날짜는 2025년 11월 6일입니다.",
        "refusal": null,
        "role": "assistant",
        "annotations": [],
        "audio": null,
        "function_call": null,
        "tool_calls": null
      }
    }
  ],
  "created": 1762404249,
  "model": "gpt-4.1-mini-2025-04-14",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_4c2851f862",
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 105,
    "total_tokens": 121,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
      "cached_tokens": 0
    }
  }
}
💬AI : 오늘 날짜는 2025년 11월 